In [12]:
import cv2
import os
import numpy as np
import glob
import math
import scipy
from scipy.spatial import distance
from scipy import signal
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn import metrics

In [13]:
##Lấy danh sách mẫu mống mắt
cwd = os.getcwd()
for dir_path, dir_names, file_names in os.walk(cwd):
    for f in file_names:
        list_file = open("list_file.txt","a+")
        temp=str(os.path.join(dir_path,f)+'\n')
        if '.bmp' in temp:
            list_file.write(temp)
list_file = open("list_file.txt","r")
images_train=[]
images_test=[]
ID_images_train=[]
ID_images_test=[]
if list_file.mode == 'r':
    data =list_file.readlines()
    for i in data[200:300]:
        if i[-6:-5]=='1':
            ID_images_test.append(i[-10:-5])
            images_test.append(cv2.imread(i[:-1]))
        else:
            ID_images_train.append(i[-10:-5])
            images_train.append(cv2.imread(i[:-1]))

In [14]:
print(ID_images_test)

['041_1', '042_1', '043_1', '044_1', '045_1', '046_1', '047_1', '048_1', '049_1', '050_1', '051_1', '052_1', '053_1', '054_1', '055_1', '056_1', '057_1', '058_1', '059_1', '060_1']


In [15]:
print(ID_images_train)

['041_2', '041_3', '041_4', '041_5', '042_2', '042_3', '042_4', '042_5', '043_2', '043_3', '043_4', '043_5', '044_2', '044_3', '044_4', '044_5', '045_2', '045_3', '045_4', '045_5', '046_2', '046_3', '046_4', '046_5', '047_2', '047_3', '047_4', '047_5', '048_2', '048_3', '048_4', '048_5', '049_2', '049_3', '049_4', '049_5', '050_2', '050_3', '050_4', '050_5', '051_2', '051_3', '051_4', '051_5', '052_2', '052_3', '052_4', '052_5', '053_2', '053_3', '053_4', '053_5', '054_2', '054_3', '054_4', '054_5', '055_2', '055_3', '055_4', '055_5', '056_2', '056_3', '056_4', '056_5', '057_2', '057_3', '057_4', '057_5', '058_2', '058_3', '058_4', '058_5', '059_2', '059_3', '059_4', '059_5', '060_2', '060_3', '060_4', '060_5']


In [16]:
def ImageEnhancement(normalized):
    enhanced=[]
    for res in normalized:
        res = res.astype(np.uint8)
        im=cv2.equalizeHist(res)
        enhanced.append(im)
    return enhanced

In [17]:
#modulating function as defined in paper
def m(x ,y, f):
    val = np.cos(2*np.pi*f*math.sqrt(x **2 + y**2))
    return val
#spatial filter as defined in paper
def gabor(x, y, dx, dy, f):
    gb = (1/(2*math.pi*dx*dy))*np.exp(-0.5*(x**2 / dx**2 + y**2 / dy**2)) * m(x, y, f)
    return gb

#function to calculate spatial filter over 8x8 blocks
def spatial(f,dx,dy):
    sfilter=np.zeros((8,8))
    for i in range(8):
        for j in range(8):
            sfilter[i,j]=gabor((-4+j),(-4+i),dx,dy,f)
    return sfilter

def get_vec(convolvedtrain1,convolvedtrain2):
    feature_vec=[]
    for i in range(6):
            for j in range(64):
                #Run 8 by 8 filtered block iteratively over the entire image
                start_height = i*8
                end_height = start_height+8
                start_wid = j*8
                end_wid = start_wid+8
                grid1 = convolvedtrain1[start_height:end_height, start_wid:end_wid]
                grid2 = convolvedtrain2[start_height:end_height, start_wid:end_wid]

                # Channel 1
                absolute = np.absolute(grid1)
                # mean
                mean = np.mean(absolute)
                feature_vec.append(mean)
                #deviation
                std = np.mean(np.absolute(absolute-mean))
                feature_vec.append(std)

                # Channel 2
                absolute = np.absolute(grid2)
                # mean
                mean = np.mean(absolute)
                feature_vec.append(mean)
                #deviation
                std = np.mean(np.absolute(absolute-mean))
                feature_vec.append(std)

    return feature_vec

def FeatureExtraction(enhanced):
    con1=[]
    con2=[]
    #get spatial filters
    filter1=spatial(0.67,3,1.5)
    filter2=spatial(0.67,4,1.5) 
    
    feature_vector=[]
    
    for i in range(len(enhanced)):
        img=enhanced[i]
        #define a 48x512 region over which the filters are applied
        img_roi=img[:48,:]
        
        filtered1=scipy.signal.convolve2d(img_roi,filter1,mode='same')
        filtered2=scipy.signal.convolve2d(img_roi,filter2,mode='same')
        
        con1.append(filtered1)
        con2.append(filtered2)
        fv=get_vec(filtered1,filtered2)
        feature_vector.append(fv)
    return feature_vector #each feature vector has a dimension of 1536



In [18]:
images_train=[ImageEnhancement(i) for i in images_train]
train=[FeatureExtraction(i) for i in images_train]

In [19]:
images_test=[ImageEnhancement(i) for i in images_test]
test=[FeatureExtraction(i) for i in images_test]

In [20]:
feature_vector_test=test
feature_vector_train=train

In [21]:
def score(feature_vector_train,feature_vector_test):
    score=0
    for i in range(0,1536):
        if np.isnan(feature_vector_train[i])==True and np.isnan(feature_vector_test[i])==True:
            score+=0
        elif np.isnan(feature_vector_train[i])==True:
            score+=feature_vector_test[i]
        elif np.isnan(feature_vector_test[i])==True:
            score+=feature_vector_train[i]
        else:
            score+=abs(feature_vector_train[i]-feature_vector_test[i])
    return score

In [22]:
a=[[sum([score(feature_vector_train[i][j],feature_vector_test[k][j]) for j in range(0,48)]) for i in range(0,80)] for k in range(0,20)]

In [23]:
a

[[343.9137268575674,
  288.08959634873554,
  346.4081454905016,
  392.0465640339646,
  870.6466065384529,
  811.3430773229655,
  822.5771895084833,
  787.3847686742778,
  601.1024687743536,
  613.1533544349293,
  633.3490944477567,
  680.8621766281921,
  818.9402743688058,
  801.420456937011,
  817.0388334349351,
  789.9423694729288,
  870.1562274265839,
  793.8406144520923,
  761.1812869567509,
  795.0799797747823,
  1025.7062065175967,
  921.4142869109083,
  924.1387477377014,
  850.8342974075292,
  875.8014497780009,
  913.3265172887486,
  943.9307082471794,
  948.6794672714398,
  1336.21597574349,
  1305.2598251804616,
  1280.8300550689726,
  1322.0214595358768,
  720.8208163392865,
  717.9201383325599,
  730.5595304099477,
  710.2563690024067,
  909.6218785629969,
  896.3585588867506,
  912.3906025636203,
  973.9566710915265,
  784.6587682084385,
  765.245346632054,
  788.8211236208645,
  769.3291672407374,
  867.0100312343301,
  813.153628789681,
  949.5953948971506,
  927.892150

In [24]:
result=[ID_images_train[i.index(min(i))][0:3] for i in a]

In [25]:
print(result)

['041', '056', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060']


In [26]:
ID_images_test=[i[0:3] for i in ID_images_test ]

In [27]:
print(ID_images_test)

['041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060']


In [28]:
from scipy.spatial import distance

In [29]:
1-distance.hamming(result,ID_images_test)

0.95